In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/dna/week6

/content/drive/MyDrive/Colab Notebooks/dna/week6


In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/dna/week6'

#1.학습한 가중치 불러오기

In [ ]:
import torch
state_dict = torch.load('resnet50.pth')
best_epoch = state_dict['epoch']
best_test_acc = state_dict['test_acc']
weights = state_dict['net']

print(f'최종적으로 {best_epoch}번째 에포크에서 test셋 기준으로 {best_test_acc}를 달성하였습니다.')#변수로 바꾸기기

최종적으로 99번째 에포크에서 test셋 기준으로 3181.279815047941를 달성하였습니다.


In [ ]:
# state_dict['net']

#2. 학습한 가중치를 모델에 불러오기

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device 배정
torch.manual_seed(42)
if device == 'cuda':
  torch.cuda.manual_seed_all(42)
device

'cuda'

In [ ]:
from torchvision import models # 모델 라이브러리 함수

resnet_50 = models.resnet50(pretrained=False).to(device) # 선행학습 여부 , finetunig한 부분이 있으니까까

# finetuning
import torch.nn as nn # 파이토치 뉴럴네트워크 layer 라이브러리
resnet_50.fc = nn.Linear(resnet_50.fc.in_features, 3).to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
#학습한 가중치 적용완료료
resnet_50.load_state_dict(weights)

<All keys matched successfully>

#3. test셋의 최종 성능 확인하기
과제
> 모델은 100 epochs를 돌려서 만든 가중치를 이용함.            
> test_loader를 생성하여, test_set의 최종 성능 평가        
> 텐서보드는 자율로 이용하시오

In [ ]:
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from dataset import Custom_dataset as C


# from dataset import Custom_dataset as C
# from torch.utils.data import Dataset, DataLoader
# import cv2
# import os 
# import torch
# import torchvision
# from torchvision import transforms # 이미지 데이터 augmentation
# import glob
# import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2 # albumentations 텐서화 함수


root_path = '/content/drive/MyDrive/Colab Notebooks/dna/week6/original-1'

test_transforms = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 텐서타입은 안해줌
    ToTensorV2() # Normalize를 먼저하고 tensor화를 진행해야한다.
])
test_class = C(root_path=root_path, mode='test', transforms=test_transforms)
test_loader = DataLoader(test_class, batch_size=4, shuffle = False, num_workers=0)

In [ ]:
test_loader

In [ ]:
model = resnet_50

model.eval()
test_loss = 0
correct = 0
criterion =  nn.CrossEntropyLoss(reduction='sum') #add all samples in a mini-batch
with torch.no_grad():
  for test_img, test_label in test_loader:
    test_img, test_label = test_img.to(device), test_label.to(device)
    output = resnet_50(test_img) #모델에 입력
    loss = criterion(output, test_label)
    test_loss +=  loss.item()
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(test_label.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
  test_loss, correct, len(test_loader.dataset),
  100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.7384, Accuracy: 110/134 (82%)

